# GWASdb-Disease

Author: Moshe Silverstein <br/>
Date: 11-17 <br/>
Data Source: http://jjwanglab.org/gwasdb

In [1]:
import sys, datetime, os
import numpy as np
import pandas as pd
import importlib
import untility_functions as uf
%matplotlib inline

In [2]:
importlib.reload(uf)

<module 'untility_functions' from '/Users/moshesilverstein/Documents/Harmonizome/GWASdb/untility_functions.py'>

# Load Data

In [6]:
df = pd.read_csv('Input/gwasdb_20150819_snp_trait.gz', sep='\t')

/Users/moshesilverstein/anaconda/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [7]:
df.head()

,CHR,POS,SNPID,REF,ALT,ORI_SNPID,PMID,P_VALUE,P_VALUE_TEXT,OR/BETA,...,MESH_TERM,EFO_ID,EFO_TERM,DOLITE_TERM,RISK_ALLELE,PUBLICATION_TYPE,AA,GENE_SYMBOL,TYPE,REFGENE
0,1,565596,rs568632519,G,A,rs28357983,22091778,7.280000e-05,Taste change in healthy volunteers in response...,NaN,...,NaN,NaN,NaN,Mental retardation,NaN,"Research Support, N.I.H., Extramural",NaN,NaN,NaN,NaN
1,1,807512,rs10751454,A,G,rs10751454,22174011,2.200000e-13,Cardiovascular disease prevalence,NaN,...,NaN,NaN,NaN,Cancer|Cardiovascular disease,NaN,"Research Support, N.I.H., Extramural|Research ...",G,FAM41C,intron,NR_027055
2,1,807512,rs10751454,A,G,rs10751454,22174011,5.400000e-21,Systolic blood pressure (SBP),NaN,...,NaN,NaN,NaN,Cancer|Cardiovascular disease,NaN,"Research Support, N.I.H., Extramural|Research ...",G,FAM41C,intron,NR_027055
3,1,807512,rs10751454,A,G,rs10751454,22174011,7.700000e-09,Total cholesterol,NaN,...,NaN,NaN,NaN,Cancer|Cardiovascular disease,NaN,"Research Support, N.I.H., Extramural|Research ...",G,FAM41C,intron,NR_027055
4,1,880238,rs3748592,A,G,rs3748592,24665060,1.090000e-04,NaN,NaN,...,NaN,NaN,NaN,Drug abuse,NaN,"Research Support, N.I.H., Extramural",G,NOC2L,intron,NM_015658


In [8]:
df.shape

(314237, 30)

# Get Relevent Data

In [75]:
df = df[['GENE_SYMBOL', 'DOLITE_TERM']]

In [78]:
df.head(10)

,GENE_SYMBOL,DOLITE_TERM
0,NaN,Mental retardation
1,FAM41C,Cancer|Cardiovascular disease
2,FAM41C,Cancer|Cardiovascular disease
3,FAM41C,Cancer|Cardiovascular disease
4,NOC2L,Drug abuse
5,NaN,Amyotrophic lateral sclerosis
6,NaN,Obesity|Overnutrition
7,C1orf159,Eating disorder
8,C1orf159,NaN
9,C1orf159,Eating disorder


In [79]:
df.shape

(314237, 2)

In [80]:
df.drop_duplicates(inplace=True)

In [81]:
df.shape

(57432, 2)

In [82]:
df.head()

,GENE_SYMBOL,DOLITE_TERM
0,NaN,Mental retardation
1,FAM41C,Cancer|Cardiovascular disease
4,NOC2L,Drug abuse
5,NaN,Amyotrophic lateral sclerosis
6,NaN,Obesity|Overnutrition


In [83]:
df.dropna(inplace=True)

In [84]:
df.shape

(51282, 2)

In [85]:
df.head()

,GENE_SYMBOL,DOLITE_TERM
1,FAM41C,Cancer|Cardiovascular disease
4,NOC2L,Drug abuse
7,C1orf159,Eating disorder
13,TTLL10,Drug abuse
14,TNFRSF18,Neutropenia


In [86]:
df_interactions = pd.DataFrame()

for i, index in enumerate(df.index):
    
    progressPercent = ((i+1)/len(df.index))*100

    sys.stdout.write("Progeres: %d%%  %d Out of %d   \r" % (progressPercent, (i+1), len(df.index)))
    sys.stdout.flush()
    
    if type(df.ix[index, 'GENE_SYMBOL']) != float:
        lst2 = df.ix[index, 'DOLITE_TERM'].split('|')
        lst1 = [df.ix[index, 'GENE_SYMBOL']]*(len(lst2))
        temp = pd.DataFrame()
        temp['Disease'] = lst2
        temp['Gene Name'] = lst1
        df_interactions = pd.concat([df_interactions, temp]) 

/Users/moshesilverstein/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:10: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated


In [87]:
df_interactions.head()

,Disease,Gene Name
0,Cancer,FAM41C
1,Cardiovascular disease,FAM41C
0,Drug abuse,NOC2L
0,Eating disorder,C1orf159
0,Drug abuse,TTLL10


In [88]:
df_interactions.shape

(68598, 2)

# Map Gene Symbols To Up-to-date Approved Gene Symbols

In [89]:
df_interactions.set_index('Gene Name', inplace=True)

In [90]:
uf.mapgenesymbols(df_interactions)

# Drop Duplicates

In [91]:
df_interactions.reset_index(inplace=True)

In [92]:
df_interactions.drop_duplicates(inplace=True)

In [93]:
df_interactions.shape

(60246, 2)

# Create Binary Matrix

In [94]:
binary_matrix = uf.createBinaryMatix(df_interactions)

In [95]:
binary_matrix.head()

,Panic disorder,Rheumatoid arthritis,Depression,Neuroblastoma,Vitiligo,Parkinson disease,Cholangitis,Celiac disease,Chronic obstructive airway disease,Esophagus cancer,...,Atopic rhinitis,Neurotic disorder,Chronic fatigue syndrome,Anemia,Aseptic necrosis of bone,Arthritis,Gonadal dysgenesis,Gouts,Atherosclerosis,Combined immunodeficiency
CCL25,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
WDR81,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
DPY19L1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
ID4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
NLRP2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [96]:
binary_matrix.shape

(12091, 225)

# Save Binary Matrix

In [97]:
filename = '~/./Documents/Harmonizome/GWASdb/Output/gwasdb_binary_matrix_%s.tsv.zip'% str(datetime.date.today())[0:7].replace('-', '_')
binary_matrix.to_csv(filename, sep='\t', compression='gzip')

# Create Gene Set Library

In [98]:
path = '/Users/moshesilverstein/Documents/Harmonizome/GWASdb/Output/'

In [99]:
name = 'gwasdb_gene_set'

In [100]:
uf.createUpGeneSetLib(binary_matrix, path, name)

# Create Attribute Library

In [102]:
path = '/Users/moshesilverstein/Documents/Harmonizome/GWASdb/Output/'

In [101]:
name = 'gwasdb_attribute_set'

In [103]:
uf.createUpAttributeSetLib(binary_matrix, path, name)

# Create Gene Similarity Matrix

In [104]:
gene_similarity_matix = uf.createSimilarityMatrix(binary_matrix, 'jaccard')

In [105]:
gene_similarity_matix.head()

,CCL25,WDR81,DPY19L1,ID4,NLRP2,LOXL2,ZMYND15,ZMIZ2,AIFM3,NUP210L,...,BLOC1S5,SLCO2B1,CWC15,LMNTD1,HCG22,DNAJC3,RSPO2,CNTN3,NACA,TBRG4
CCL25,1.0,0.0,0.0,0.0,0.0,0.000000,0.142857,0.0,0.0,0.000000,...,0.0,0.0,0.000,0.0,0.0,0.000000,0.090909,0.000000,0.000000,0.333333
WDR81,0.0,1.0,0.0,0.4,0.0,0.222222,0.000000,0.0,0.0,0.000000,...,0.0,0.0,0.125,0.0,0.0,0.000000,0.000000,0.142857,0.000000,0.000000
DPY19L1,0.0,0.0,1.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,...,0.0,0.0,0.000,0.0,0.0,0.000000,0.100000,0.000000,0.000000,0.000000
ID4,0.0,0.4,0.0,1.0,0.0,0.333333,0.000000,0.0,0.0,0.000000,...,0.0,0.0,0.200,0.0,0.0,0.000000,0.000000,0.083333,0.000000,0.000000
NLRP2,0.0,0.0,0.0,0.0,1.0,0.000000,0.100000,0.0,0.0,0.166667,...,0.0,0.0,0.000,0.0,0.0,0.166667,0.071429,0.000000,0.166667,0.000000


# Save Gene Similarity Matrix

In [106]:
filename = '~/./Documents/Harmonizome/GWASdb/Output/gwasdb_gene_similarity_matix_%s.tsv.zip'% str(datetime.date.today())[0:7].replace('-', '_')
gene_similarity_matix.to_csv(filename, sep='\t', compression='gzip')

# Create Attribute Similarity matrix

In [107]:
attribute_similarity_matix = uf.createSimilarityMatrix(binary_matrix.T, 'jaccard')

In [108]:
attribute_similarity_matix.head()

,Panic disorder,Rheumatoid arthritis,Depression,Neuroblastoma,Vitiligo,Parkinson disease,Cholangitis,Celiac disease,Chronic obstructive airway disease,Esophagus cancer,...,Atopic rhinitis,Neurotic disorder,Chronic fatigue syndrome,Anemia,Aseptic necrosis of bone,Arthritis,Gonadal dysgenesis,Gouts,Atherosclerosis,Combined immunodeficiency
Panic disorder,1.000000,0.015924,0.022287,0.006849,0.013333,0.021040,0.017778,0.020979,0.028689,0.008097,...,0.000000,0.010753,0.000000,0.000000,0.000000,0.011952,0.000000,0.013575,0.011029,0.000000
Rheumatoid arthritis,0.015924,1.000000,0.095238,0.010846,0.035359,0.077644,0.027356,0.071642,0.067164,0.026812,...,0.000000,0.002281,0.008939,0.001126,0.001140,0.033831,0.001145,0.016113,0.065520,0.000000
Depression,0.022287,0.095238,1.000000,0.011788,0.015702,0.090677,0.026777,0.031607,0.059679,0.026292,...,0.001025,0.002051,0.005020,0.000000,0.003083,0.038356,0.000000,0.019337,0.080329,0.000000
Neuroblastoma,0.006849,0.010846,0.011788,1.000000,0.007937,0.010101,0.004926,0.007547,0.010593,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.004386,0.000000,0.010152,0.007557,0.016393
Vitiligo,0.013333,0.035359,0.015702,0.007937,1.000000,0.006250,0.029557,0.046154,0.014737,0.017778,...,0.000000,0.013699,0.010309,0.000000,0.000000,0.008621,0.000000,0.014925,0.015132,0.000000


# Save Attribute Similarity Matrix

In [109]:
filename = '~/./Documents/Harmonizome/GWASdb/Output/gwasdb_attribute_similarity_matix_%s.tsv.zip'% str(datetime.date.today())[0:7].replace('-', '_')
attribute_similarity_matix.to_csv(filename, sep='\t', compression='gzip')

# Create Gene List

In [110]:
gene_list = uf.createGeneList(binary_matrix)

In [111]:
gene_list.head()

,GeneSym,GeneID
0,CCL25,6370
1,WDR81,124997
2,DPY19L1,23333
3,ID4,3400
4,NLRP2,55655


In [112]:
gene_list.shape

(12091, 2)

# Save Gene List

In [113]:
filename = '~/./Documents/Harmonizome/GWASdb/Output/gwasdb_gene_list_%s.tsv.zip'% str(datetime.date.today())[0:7].replace('-', '_')
gene_list.to_csv(filename, sep='\t', index=False, compression='gzip')

# Create Attribute List

In [114]:
attribute_list = uf.createAttributeList(binary_matrix)

In [115]:
attribute_list.head()

,Attributes
0,Panic disorder
1,Rheumatoid arthritis
2,Depression
3,Neuroblastoma
4,Vitiligo


In [116]:
attribute_list.shape

(225, 1)

# Save Attribute List

In [117]:
filename = '~/./Documents/Harmonizome/GWASdb/Output/gwasdb_attribute_list_%s.tsv.zip'% str(datetime.date.today())[0:7].replace('-', '_')
attribute_list.to_csv(filename, sep='\t', index=False, compression='gzip')

# Create Gene-Attribute Edge List

In [118]:
path = '/Users/moshesilverstein/Documents/Harmonizome/GWASdb/Output/'

In [119]:
name = 'gwasdb_gene_attribute_edge_list'

In [120]:
uf.createGeneAttributeEdgeList(binary_matrix, gene_list, path, name)

Progeres: 100%  225 Out of 225   

 The number of statisticaly relevent gene-attribute associations is: 60246
